
# X-band Space-to-Earth PFD vs ITU Mask (8025-8400 MHz)

This notebook computes power-flux density (PFD) at the Earth's surface from a LEO spacecraft downlink and compares it against the ITU Radio Regulations Article 21 PFD mask (per 4 kHz) as a function of arrival elevation.


In [ ]:

# Reusable PFD vs Elevation sweep
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dataclasses import dataclass
from typing import Tuple

@dataclass
class LinkParams:
    """Parameters for the downlink and geometry."""
    freq_hz: float                 # Carrier frequency (Hz)
    tx_power_w: float              # RF power at PA output (W)
    ant_gain_dbi: float            # Antenna gain toward Earth (dBi)
    feed_losses_db: float          # Interconnect/feed losses (dB, positive means loss)
    occupied_bw_hz: float          # Occupied bandwidth (Hz)
    alt_m: float                   # Orbit altitude (m)

EARTH_RADIUS_M = 6371e3

def slant_range_m(elev_deg: float, alt_m: float) -> float:
    """Geometric slant range from ground to satellite at elevation 'elev_deg'."""
    e = math.radians(elev_deg)
    Re = EARTH_RADIUS_M
    h = alt_m
    return math.sqrt((Re + h)**2 - (Re * math.cos(e))**2) - Re * math.sin(e)

def itu_rr_mask_dbw_m2_per_4k(elev_deg: float) -> float:
    """ITU RR Art.21 mask (8025-8400 MHz, space-to-Earth), per 4 kHz, vs. elevation."""
    d = elev_deg
    if d <= 5.0:
        return -150.0
    elif d <= 25.0:
        return -150.0 + 0.5 * (d - 5.0)
    else:
        return -140.0

def pfd_at_surface_dbw_m2_per_4k(elev_deg: float, lp: LinkParams):
    """Return (EIRP_density_4k_dBW, PFD_dBW_m2_4k, slant_range_m)."""
    tx_power_dbw = 10 * math.log10(lp.tx_power_w)
    eirp_dbw = tx_power_dbw + lp.ant_gain_dbi - lp.feed_losses_db
    eirp_density_4k_dbw = eirp_dbw - 10 * math.log10(lp.occupied_bw_hz / 4000.0)
    R = slant_range_m(elev_deg, lp.alt_m)
    spreading_db = 10 * math.log10(4 * math.pi * R * R)
    pfd_dbw_m2_4k = eirp_density_4k_dbw - spreading_db
    return eirp_density_4k_dbw, pfd_dbw_m2_4k, R

def sweep_elevation(lp: LinkParams, start_deg: float = 0.0, stop_deg: float = 90.0, step_deg: float = 0.5) -> pd.DataFrame:
    elevs = np.arange(start_deg, stop_deg + 1e-6, step_deg)
    rows = []
    for d in elevs:
        eirp_den, pfd, R = pfd_at_surface_dbw_m2_per_4k(d, lp)
        mask = itu_rr_mask_dbw_m2_per_4k(d)
        margin = mask - pfd
        rows.append({
            "Elevation (deg)": d,
            "Slant range (km)": R / 1000.0,
            "EIRP density (dBW/4kHz)": eirp_den,
            "PFD (dBW/m^2/4kHz)": pfd,
            "ITU mask (dBW/m^2/4kHz)": mask,
            "Margin to mask (dB)": margin,
        })
    return pd.DataFrame(rows)

# Example run with your parameters
lp = LinkParams(
    freq_hz=8.22e9,
    tx_power_w=2.0,
    ant_gain_dbi=17.0,       # Use ~20.2 for the PCLL 4x4 (approx 17.2 dBic + 3 dB)
    feed_losses_db=0.5,
    occupied_bw_hz=30e6,
    alt_m=500e3,
)

df = sweep_elevation(lp)
df.head(10)


In [ ]:

# Plot PFD vs ITU mask
plt.figure()
plt.plot(df["Elevation (deg)"], df["PFD (dBW/m^2/4kHz)"], label="PFD at surface")
plt.plot(df["Elevation (deg)"], df["ITU mask (dBW/m^2/4kHz)"], linestyle="--", label="ITU RR mask")
plt.xlabel("Elevation (deg)")
plt.ylabel("Level (dBW/m^2/4 kHz)")
plt.title("Space-to-Earth PFD vs ITU Mask (8025-8400 MHz)")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Plot compliance margin
plt.figure()
plt.plot(df["Elevation (deg)"], df["Margin to mask (dB)"], label="Margin (mask - PFD)")
plt.axhline(0, linestyle="--")
plt.xlabel("Elevation (deg)")
plt.ylabel("Margin (dB)")
plt.title("Compliance Margin vs Elevation")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Save results to CSV
csv_path = "xband_pfd_vs_elevation.csv"
df.to_csv(csv_path, index=False)
csv_path


In [ ]:

# --- PCLL 4x4 patch antenna case (approx 17.2 dBic -> ~20.2 dBi at 8.22 GHz) ---
lp_pcll = LinkParams(
    freq_hz=8.22e9,
    tx_power_w=2.0,
    ant_gain_dbi=20.2,   # 17.2 dBic + 3 dB
    feed_losses_db=0.5,
    occupied_bw_hz=30e6,
    alt_m=500e3,
)

df_pcll = sweep_elevation(lp_pcll)
df_pcll.head(10)


In [ ]:

# Compare PFD vs mask for both antennas
plt.figure()
plt.plot(df["Elevation (deg)"], df["PFD (dBW/m^2/4kHz)"], label="PFD (17 dBi)")
plt.plot(df_pcll["Elevation (deg)"], df_pcll["PFD (dBW/m^2/4kHz)"], label="PFD (PCLL ~20.2 dBi)")
plt.plot(df["Elevation (deg)"], df["ITU mask (dBW/m^2/4kHz)"], linestyle="--", label="ITU RR mask")
plt.xlabel("Elevation (deg)")
plt.ylabel("Level (dBW/m^2/4 kHz)")
plt.title("PFD vs ITU Mask — Baseline vs PCLL 4x4")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Compare compliance margin for both antennas
plt.figure()
plt.plot(df["Elevation (deg)"], df["Margin to mask (dB)"], label="Margin (17 dBi)")
plt.plot(df_pcll["Elevation (deg)"], df_pcll["Margin to mask (dB)"], label="Margin (PCLL ~20.2 dBi)")
plt.axhline(0, linestyle="--")
plt.xlabel("Elevation (deg)")
plt.ylabel("Margin (dB)")
plt.title("Compliance Margin vs Elevation — Baseline vs PCLL 4x4")
plt.legend()
plt.grid(True)
plt.show()


In [ ]:

# Save PCLL results
csv_path_pcll = "xband_pfd_vs_elevation_PCLL.csv"
df_pcll.to_csv(csv_path_pcll, index=False)
csv_path_pcll


In [ ]:

# Plot only the ITU RR mask vs elevation for clarity
elevs = np.linspace(0, 90, 181)
mask_vals = [itu_rr_mask_dbw_m2_per_4k(d) for d in elevs]

plt.figure()
plt.plot(elevs, mask_vals, label="ITU RR mask")
plt.xlabel("Elevation (deg)")
plt.ylabel("Mask level (dBW/m^2/4 kHz)")
plt.title("ITU RR Article 21 PFD Mask vs Elevation (8025-8400 MHz)")
plt.grid(True)
plt.legend()
plt.show()


In [ ]:

# Plot the ITU RR mask alone vs elevation (0-90 deg)
import numpy as np
import matplotlib.pyplot as plt

elev = np.linspace(0, 90, 361)
def itu_rr_mask_dbw_m2_per_4k(elev_deg: float) -> float:
    if elev_deg <= 5.0:
        return -150.0
    elif elev_deg <= 25.0:
        return -150.0 + 0.5 * (elev_deg - 5.0)
    else:
        return -140.0

mask_vals = np.array([itu_rr_mask_dbw_m2_per_4k(d) for d in elev])

plt.figure()
plt.plot(elev, mask_vals, linestyle="-", label="ITU RR mask (8025-8400 MHz)")
plt.xlabel("Elevation (deg)")
plt.ylabel("PFD limit (dBW/m^2/4 kHz)")
plt.title("ITU RR Article 21 PFD Mask vs Elevation")
plt.grid(True)
plt.legend()
plt.show()
